In [4]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import os
import pandas as pd
import re

from util2.tools import insert_text_after, up_ten

In [ ]:
# NA 지역 Trip Adviser의 모든 호텔 긁어버림.
total_result = []
chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

for i in range(0, 690, 30):
    if i == 0:
        url = 'https://www.tripadvisor.com/Hotels-g19-zfd21371-a_ufe.true-North_America-Hotels.html'
    else:
        url = (f'https://www.tripadvisor.com/Hotels-g19-oa{i}-zfd21371-a_ufe.true-North_America-Hotels.html')
    
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    item = soup.select('a.BMQDV._F.Gv.wSSLS.SwZTJ.FGwzt.ukgoS')
    
    for text in item:
        text = str(text)
        text = text.split('Gv wSSLS SwZTJ FGwzt ukgoS" href="')
        if len(text) == 2:
            text = text[1]
        else:
            print('Split Error, Result Length = {}, Need to Check start_str' .format(len(text)))
        text = text.split('" target="_blank"><')    
        if len(text) == 2:
            text = text[0]
        else:
            print('Split Error, Result Length = {}, Need to Check end_str' .format(len(text)))
        text = 'https://www.tripadvisor.com' + text
        total_result.append(text)
        
    total_result = list(set(total_result))
    print(len(total_result))
    
df = pd.DataFrame()
for num, i in enumerate(total_result):
    df.loc[num, 'url'] = i
    
df.to_csv('./NA_hotel.csv', index=False)

In [ ]:
# 추출된 URL에 접속해서 특정 데이터들을 가져올 수 있는지 시도. 일단 가능함.
total_result = []
chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

for num, url in enumerate(df.url):
    driver.get(url)
    time.sleep(2)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    df.loc[num, 'hotel_name'] = soup.find('h1', id='HEADING').text
    df.loc[num, 'location'] = soup.find_all('span', class_='biGQs _P pZUbB KxBGd')[1].text

    span_tag = soup.find('span', class_='biGQs _P pZUbB KxBGd')
    df.loc[num, 'num_review'] = int(re.search(r'(\d+)', span_tag.text).group(1))
    df.loc[num, 'score'] = float(soup.find('span', class_='uwJeR P').text)
    
    print(df.iloc[num, :])
    

In [ ]:
# 해당 URL에 접속해서 모든 리뷰 - 답변 데이터를 가져옴. 없는 것은 'No Exist'라고 처리. 나중에 제거하거나 뭐 .. 아무튼.
df = pd.read_csv('./NA_hotel_2.csv')

chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
time.sleep(1)

for num in range(696):
    url = df.url[num]
    num_review = int(df.num_review[num])
    limit_num = up_ten(num_review)
    
    print(f'Start. Hotel Number = {num}, Review Number = {num_review}')
    try:
        if num_review < 10:
            temp_df = pd.DataFrame()
            driver.get(url)
            time.sleep(1)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            result = soup.find_all('div', class_='vTVDc')
            
            for idx in range(num_review):
                try:
                    temp_df.loc[idx, 'user_review'] = result[idx].find('span', class_='QewHA H4 _a').text
                except:
                    temp_df.loc[idx, 'user_review'] = 'No Exist'
                
                try:
                    temp_df.loc[idx, 'response'] = result[idx].find('span', class_='MInAm _a').text
                except:
                    temp_df.loc[idx, 'response'] = 'No Exist'
                
            temp_df.to_csv(f'./Data/Hotel_data/Hotel_Review_csv/hotel_{num}.csv', index=False)
            temp_df.to_excel(f'./Data/Hotel_data/Hotel_Review_Excel/hotel_{num}.xlsx', index=False)
            print(f'Save_Done. Hotel Number = {num}')
                
        else:
            temp_df = pd.DataFrame()
            total_idx = 0
            for i in range(0, limit_num, 10):
                if i == 0:
                    driver.get(url)
                else:
                    page_num = (f'or{i}-')
                    temp_url = insert_text_after(url, "-Reviews-", page_num)
                    driver.get(temp_url)
                    print(f'Hotel_number = {num}, # of review = {i}')
                    
                time.sleep(1)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                result = soup.find_all('div', class_='vTVDc')
                
                for idx in range(len(result)):
                    try:
                        temp_df.loc[total_idx, 'user_review'] = result[idx].find('span', class_='QewHA H4 _a').text
                    except:
                        temp_df.loc[total_idx, 'user_review'] = 'No Exist'
                    
                    try:
                        temp_df.loc[total_idx, 'response'] = result[idx].find('span', class_='MInAm _a').text
                    except:
                        temp_df.loc[total_idx, 'response'] = 'No Exist'
                        
                    total_idx = (total_idx + 1)
                    
            temp_df.to_csv(f'./Data/Hotel_data/Hotel_Review_csv/hotel_{num}.csv', index=False)
            temp_df.to_excel(f'./Data/Hotel_data/Hotel_Review_Excel/hotel_{num}.xlsx', index=False)
            print(f'Save_Done. Hotel Number = {num}')
    except:
        print(f'Error. Hotel Number = {num}')

In [ ]:
# 해당 URL에 접속해서 모든 리뷰 - 답변 데이터를 가져옴. 없는 것은 'No Exist'라고 처리. 나중에 제거하거나 뭐 .. 아무튼.
df = pd.read_csv('./NA_hotel_2.csv')

chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
time.sleep(1)

for num in range(696):
    url = df.url[num]
    num_review = int(df.num_review[num])
    limit_num = up_ten(num_review)
    
    print(f'Start. Hotel Number = {num}, Review Number = {num_review}')
    try:
        if num_review < 10:
            temp_df = pd.DataFrame()
            driver.get(url)
            time.sleep(1)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            result = soup.find_all('div', class_='WAllg _T')
            
            for idx in range(num_review):
                try:
                    temp_df.loc[idx, 'user_review'] = result[idx].find('span', class_='QewHA H4 _a').text
                except:
                    temp_df.loc[idx, 'user_review'] = 'No Exist'
                
                try:
                    rating_tag = result[idx].find('span', class_=re.compile(r'ui_bubble_rating bubble_\d+'))
                    temp_df.loc[idx, 'user_rating'] = (int(re.search(r'bubble_(\d+)', rating_tag['class'][1]).group(1))) / 10
                except:
                    temp_df.loc[idx, 'user_rating'] = 'No Exist'

                try:
                    temp_df.loc[idx, 'response'] = result[idx].find('span', class_='MInAm _a').text
                except:
                    temp_df.loc[idx, 'response'] = 'No Exist'
                
            temp_df.to_csv(f'./Data/Hotel_data/Hotel_Review_csv/hotel_{num}.csv', index=False)
            temp_df.to_excel(f'./Data/Hotel_data/Hotel_Review_Excel/hotel_{num}.xlsx', index=False)
            print(f'Save_Done. Hotel Number = {num}')
                
        else:
            temp_df = pd.DataFrame()
            total_idx = 0
            for i in range(0, limit_num, 10):
                if i == 0:
                    driver.get(url)
                else:
                    page_num = (f'or{i}-')
                    temp_url = insert_text_after(url, "-Reviews-", page_num)
                    driver.get(temp_url)
                    print(f'Hotel_number = {num}, # of review = {i}')
                    
                time.sleep(1)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                result = soup.find_all('div', class_='WAllg _T')
                
                for idx in range(len(result)):
                    try:
                        temp_df.loc[total_idx, 'user_review'] = result[idx].find('span', class_='QewHA H4 _a').text
                    except:
                        temp_df.loc[total_idx, 'user_review'] = 'No Exist'
                        
                    try:
                        rating_tag = result[idx].find('span', class_=re.compile(r'ui_bubble_rating bubble_\d+'))
                        temp_df.loc[total_idx, 'user_rating'] = (int(re.search(r'bubble_(\d+)', rating_tag['class'][1]).group(1))) / 10
                    except:
                        temp_df.loc[total_idx, 'user_rating'] = 'No Exist'
                    
                    try:
                        temp_df.loc[total_idx, 'response'] = result[idx].find('span', class_='MInAm _a').text
                    except:
                        temp_df.loc[total_idx, 'response'] = 'No Exist'
                        
                    total_idx = (total_idx + 1)
                    
            temp_df.to_csv(f'./Data/Hotel_data/Hotel_Review_csv/hotel_{num}.csv', index=False)
            temp_df.to_excel(f'./Data/Hotel_data/Hotel_Review_Excel/hotel_{num}.xlsx', index=False)
            print(f'Save_Done. Hotel Number = {num}')
    except:
        print(f'Error. Hotel Number = {num}')